<a href="https://colab.research.google.com/github/Gulshan89765/LandmarkDetection/blob/main/Snippets_Importing_libraries.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing a library that is not in Colaboratory

To import a library that's not in Colaboratory by default, you can use `!pip install` or `!apt-get install`.

In [21]:
!pip install matplotlib-venn

In [22]:
!apt-get -qq install -y libfluidsynth1

E: Package 'libfluidsynth1' has no installation candidate


# Install 7zip reader [libarchive](https://pypi.python.org/pypi/libarchive)

In [23]:
# https://pypi.python.org/pypi/libarchive
!apt-get -qq install -y libarchive-dev && pip install -U libarchive
import libarchive

# Install GraphViz & [PyDot](https://pypi.python.org/pypi/pydot)

In [24]:
# https://pypi.python.org/pypi/pydot
!apt-get -qq install -y graphviz && pip install pydot
import pydot

# Install [cartopy](http://scitools.org.uk/cartopy/docs/latest/)

In [25]:
!pip install cartopy
import cartopy

In [26]:
# Install required packages
!pip install -q streamlit langchain openai faiss-cpu pymupdf tiktoken
!pip install -U langchain-community

# File: genai_assistant.py
import fitz  # PyMuPDF
import streamlit as st
from langchain.text_splitter import CharacterTextSplitter
# Update import for OpenAIEmbeddings as it has moved to langchain_community
from langchain_community.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA
import tempfile
import os

# Sidebar: Upload document
st.sidebar.title("📄 Upload Document")
uploaded_file = st.sidebar.file_uploader("Choose a PDF file", type=["pdf"])

if uploaded_file:
    # Save uploaded file temporarily
    temp_path = os.path.join(tempfile.gettempdir(), uploaded_file.name)
    with open(temp_path, "wb") as f:
        f.write(uploaded_file.read())

    # Extract text
    doc = fitz.open(temp_path)
    text = ""
    for page in doc:
        text += page.get_text()

    # Split text into chunks
    splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
    chunks = splitter.split_text(text)

    # Create embeddings and vector store
    # The class remains the same, just the import path changes
    embeddings = OpenAIEmbeddings()
    vectordb = FAISS.from_texts(chunks, embedding=embeddings)

    # RetrievalQA setup
    qa = RetrievalQA.from_chain_type(
        llm=ChatOpenAI(model_name="gpt-3.5-turbo"),
        chain_type="stuff",
        retriever=vectordb.as_retriever(search_kwargs={"k": 3}),
        return_source_documents=True
    )

    st.title("🧠 GenAI Document Assistant")

    # User question
    question = st.text_input("Ask a question about the document:")
    if question:
        result = qa({"query": question})
        answer = result["result"]
        sources = result["source_documents"]

        st.markdown("### 📌 Answer")
        st.write(answer)

        st.markdown("### 🔍 Source Justification")
        for i, doc in enumerate(sources):
            st.markdown(f"**Source {i+1}:**")
            st.write(doc.page_content[:500] + "...")
            st.markdown("---")

    # AI-asks-you mode
    st.markdown("## 🤖 Try Logic-Based Challenge")
    if st.button("Pose Logic-Based Question"):
        logic_question = "Based on the document, what would happen if [hypothetical condition]?"
        result = qa({"query": logic_question})
        answer = result["result"]

        user_answer = st.text_input("Your answer to the logic question:")
        if user_answer:
            # Evaluate user answer using LLM
            evaluation_prompt = (
                f"Evaluate this user answer: '{user_answer}' in response to '{logic_question}'. "
                f"Is it correct based on this document context: '{text[:1500]}'. Give feedback."
            )
            # ChatOpenAI is currently still in langchain.chat_models, but keep an eye on future updates
            model = ChatOpenAI()
            eval_result = model.predict(evaluation_prompt)

            st.markdown("### ✅ AI Evaluation")
            st.write(eval_result)

2025-06-23 17:48:10.690 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-23 17:48:10.690 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-23 17:48:10.691 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-23 17:48:10.692 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-23 17:48:10.693 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-23 17:48:10.693 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-23 17:48:10.694 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-23 17:48:10.695 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar